Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# MONAI Auto3DSeg Hyper-parameter Optimization with NNI

**Auto3DSeg** supports hyperparameter optimization (HPO) with `NNI` and `Optuna` packages.
Please check the [HPO Optuna Notebook](hpo_optuna.ipynb) if you want to use **Auto3DSeg** with `Optuna` HPO.

This notebook provides an example to perform HPO on learning rate with a third-party package [NNI by Microsoft](https://nni.readthedocs.io/en/stable/).
To run this notebook, please install the package via `pip install nni`

Note: if you have used other Auto3DSeg notebooks, for example: 
- `auto_runner.ipynb`
- `auto3dseg_autorunner_ref_api.ipynb`
- `auto3dseg_hello_world.ipynb`
- `hpo_optuna.ipynb`

You may have already generated the algorithm templates in MONAI bundle formats (hint: find them in your previous working directory). 
Please feel free to skip the following steps in this tutorial:
- Download dataset
- Define experiment file paths
- Prepare an input YAML
- Create Bundle Generators

## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm, cucim, yaml, optuna, nni]"

## Setup imports

In [ ]:
import os
import yaml

import tempfile

from monai.apps import download_and_extract
from monai.apps.auto3dseg import BundleGen, DataAnalyzer, NNIGen
from monai.apps.auto3dseg.utils import export_bundle_algo_history, import_bundle_algo_history
from monai.bundle.config_parser import ConfigParser
from monai.config import print_config
from monai.utils.enums import AlgoKeys

print_config()

## Download dataset

We provide a toy datalist file that splits a subset of the downloaded datasets into five folds.

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

msd_task = "Task05_Prostate"
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/" + msd_task + ".tar"

compressed_file = os.path.join(root_dir, msd_task + ".tar")
dataroot = os.path.join(root_dir, msd_task)
if not os.path.exists(dataroot):
    download_and_extract(resource, compressed_file, root_dir)

datalist_file = os.path.join("..", "tasks", "msd", msd_task, "msd_" + msd_task.lower() + "_folds.json")

# Define experiment file paths

In [ ]:
# User created files
nni_yaml = "./nni_config.yaml"

# Experiment setup
work_dir = "./hpo_nni_work_dir"
datastats_file = os.path.join(work_dir, "datastats.yaml")
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

## Prepare an input YAML

In [ ]:
input_cfg = {
    "name": msd_task,  # optional, it is only for your own record
    "task": "segmentation",  # optional, it is only for your own record
    "modality": "MRI",  # required
    "datalist": datalist_file,  # required
    "dataroot": dataroot,  # required
}
input = "./input.yaml"
ConfigParser.export_config_file(input_cfg, input)

## Create Bundle Generators

In [ ]:
if not os.path.exists(datastats_file):
    da = DataAnalyzer(datalist_file, dataroot, output_path=datastats_file)
    da.get_all_case_stats()

# algorithm generation
bundle_generator = BundleGen(
    algo_path=work_dir,
    data_stats_filename=datastats_file,
    data_src_cfg_name=input,
)

bundle_generator.generate(work_dir, num_fold=1)
history = bundle_generator.get_history()
export_bundle_algo_history(history)

## Create Algo object from bundle_generator history

Algorithm selected to do HPO.
Refer to bundle history for the mapping between algorithm name and index.
Users can `get_history` from `bundle_generator`, or `import_bundle_algo_history` by reading bundles saved in the working directory

In [ ]:
try:
    history = bundle_generator.get_history()
    assert len(history) > 0
except Exception:
    history = import_bundle_algo_history(work_dir, only_trained=False)

print("algorithms imported from the history:")

for i, algo_dict in enumerate(history):
    print(f"{i}: ", algo_dict[AlgoKeys.ID])

## Select an algorithm to perform HPO
Note: The name of the algorithms has a convention: `{net}_{fold_index}_{other_meta_info}`

In [ ]:
for algo_dict in history:
    if algo_dict[AlgoKeys.ID].split("_")[0] == "segresnet":
        break
algo_name = algo_dict[AlgoKeys.ID]
algo = algo_dict[AlgoKeys.ALGO]
print(f"{algo_name} is selected. ")

In [ ]:
nni_gen = NNIGen(algo=algo)

## Create your NNI configs
Refer to [NNI](https://nni.readthedocs.io/en/stable/) for more details.

In [ ]:
nni_config = {
    "experimentName": msd_task + "_lr",
    "searchSpace": {"learning_rate": {"_type": "choice", "_value": [0.0001, 0.001, 0.01, 0.1]}},
    "trialCommand": None,
    "trialCodeDirectory": ".",
    "trialGpuNumber": 1,
    "trialConcurrency": 2,
    "maxTrialNumber": 10,
    "maxExperimentDuration": "1h",
    "tuner": {"name": "GridSearch"},
    "trainingService": {"platform": "local", "useActiveGpu": True},
}
with open(nni_yaml, "w") as f:
    yaml.dump(nni_config, f)

## Run NNI from terminal
### Step 1: copy the trialCommand print out info, e.g.
```
python -m monai.apps.auto3dseg NNIGen run_algo  ./hpo_nni_work_dir/segresnet2d_0/algo_object.pkl {result_dir}
```
Replace {result_dir} with a folder path to save HPO experiments.
### Step 2: copy the above trialCommand to replace the trialCommand in nni_config.yaml
### Step 3: run NNI experiemtns from a terminal with 
```
nnictl create --config ./nni_config.yaml
```

Use the print out trialCommand from NNIGen initialization to replace the trialCommand in nni_config and run NNI from terminal

## Example Results
We changed override_param to {'num_iterations':6000, 'num_iterations_per_validation':600}, to run the experiments for longer time.
Here is the results shown in NNI webui. The optimal learning rate for SegResNet2D (selected_algorithm_index=0) is 0.1, which achieves Dice score of 0.735.

![](../figures/nni_image0.png)
![](../figures/nni_image1.png)


## Override HPO parameters of existing algorithms

Users can override the parameters of choice before the HPO runs.
In such case, users need to provide an `override_param`.
The previously-generated algorithm will not be touched.
`NNIGen` will create a copy of the algorithm and save the algorithm in a new folder named `{net}_{fold_index}_override`.

For more information about creating overriding parameters, please refer to the section "Override Specific Parameters in the Algorithms before HPO" in this [tutorial documentation](../docs/hpo.md)

> NOTE: If you are using a system with more than 6 GPUs, please set the environment "CUDA_VISIBLE_DEVICES" to be "0,1,2,3,4,5".

In [ ]:
# "override_params" is used to update algorithm hyperparameters
# like num_epochs, which are not in the HPO search space. We set num_epochs=2
# to shorten the training time as an example

max_epochs = 2

# for segresnet2d
override_param = {
    "num_epochs_per_validation": 1,
    "num_epochs": max_epochs,
}

nni_gen = NNIGen(algo=algo, params=override_param)